# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [2]:
df_raw = pd.read_csv("../admissions.csv")
df = df_raw.dropna() 
df.head()

,admit,gre,gpa,prestige
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [3]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df['admit'],df['prestige'])

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [4]:
df=df.join(pd.get_dummies(df['prestige'], prefix='prestige'))

In [5]:
df.head()

,admit,gre,gpa,prestige,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380,3.61,3,0,0,1,0
1,1,660,3.67,3,0,0,1,0
2,1,800,4.00,1,1,0,0,0
3,1,640,3.19,4,0,0,0,1
4,0,520,2.93,4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We only need 3, since our base class will represent prestige 1.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [6]:
pd.crosstab(df['admit'], df['prestige'])

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


In [7]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(df['admit'], df['prestige_1.0'])

prestige_1.0,0.0,1.0
admit,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [8]:
odds_admit_teir1 =  (33./28.)
print odds_admit_teir1

1.17857142857


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [9]:
odds_noadmit_teir1 = (93./243.)
print odds_noadmit_teir1

0.382716049383


#### 3.3 Calculate the odds ratio

In [10]:
print odds_admit_teir1/odds_noadmit_teir1

3.07949308756


#### 3.4 Write this finding in a sentenance: 

Answer: You would be 3.07 times more likely to be admitted to grad school if you attended a prestige 1 school than if you hadn't.

#### 3.5 Print the cross tab for prestige_4

In [11]:
pd.crosstab(df['admit'], df['prestige_4.0'])

prestige_4.0,0.0,1.0
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [12]:
odds_admit_teir4=(12./55.)
print odds_admit_teir4

0.218181818182


In [13]:
odds_noadmit_teir4= (114./216.)
print odds_noadmit_teir4

0.527777777778


In [14]:
print odds_admit_teir4/odds_noadmit_teir4

0.413397129187


#### 3.7 Write this finding in a sentence

Answer: You are 0.41 times as likely (or 2.5 times less likely) to be admitted if you attended a prestige 4 school than if you attended another.

## Part 4. Analysis

In [15]:
# create a clean data frame for the regression
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige')
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks)
data.head()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [16]:
# manually add the intercept
data['intercept'] = 1.0
data.head()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,0,380,3.61,0,0,1,0,1
1,1,660,3.67,0,0,1,0,1
2,1,800,4.00,1,0,0,0,1
3,1,640,3.19,0,0,0,1,1
4,0,520,2.93,0,0,0,1,1


#### 4.1 Set the covariates to a variable called train_cols

In [17]:
train_cols = ['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0', "intercept"]
df_predictors = data[train_cols]
df_predictors.head()

,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,380,3.61,0,1,0,1
1,660,3.67,0,1,0,1
2,800,4.00,0,0,0,1
3,640,3.19,0,0,1,1
4,520,2.93,0,0,1,1


#### 4.2 Fit the model

In [111]:
logit = sm.Logit(data['admit'], df_predictors)
results = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [19]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sun, 03 Apr 2016   Pseudo R-squ.:                 0.08166
Time:                        21:01:25   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
intercept       -3.8769      1.142     -3.393      0.001        -6.116    -1.638
================================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [20]:
np.exp(results.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64

In [21]:
params = results.params
conf = results.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: One would be 50% less likely to be admitted to grad school if she attended a prestige 2 school compared to a student who attended a prestige 1 school.

#### 4.6 Interpret the OR of GPA

Answer: For every 1 point change in GPA, a student would be 2.18 times more likely to be admitted to grad school, compared to another student with all other factors the same.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [22]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [104]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
combos.head()

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


,0,1,2,3
0,220,2.260000,1,1
1,220,2.260000,2,1
2,220,2.260000,3,1
3,220,2.260000,4,1
4,220,2.453333,1,1


In [105]:
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']

#### 5.1 Recreate the dummy variables

In [106]:
combos.head()

,gre,gpa,prestige,intercept
0,220,2.260000,1,1
1,220,2.260000,2,1
2,220,2.260000,3,1
3,220,2.260000,4,1
4,220,2.453333,1,1


In [107]:
# recreate the dummy variables
combos_dummies = pd.get_dummies(combos['prestige'], prefix='prestige')

In [108]:
# keep only what we need for making predictions
cols_keep = ['gre', 'gpa', 'intercept']
combos = combos[cols_keep].join(combos_dummies.ix[:, 'prestige_2.0':])

In [109]:
combos.head()

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0
0,220,2.260000,1,0,0,0
1,220,2.260000,1,1,0,0
2,220,2.260000,1,0,1,0
3,220,2.260000,1,0,0,1
4,220,2.453333,1,0,0,0


#### 5.2 Make predictions on the enumerated dataset

In [112]:
combos['admit_pred'] = results.predict(combos[train_cols])

combos.tail()

,gre,gpa,intercept,prestige_2.0,prestige_3.0,prestige_4.0,admit_pred
395,800,3.806667,1,0,0,1,0.334286
396,800,4.000000,1,0,0,0,0.734040
397,800,4.000000,1,1,0,0,0.582995
398,800,4.000000,1,0,1,0,0.419833
399,800,4.000000,1,0,0,1,0.368608


#### 5.3 Interpret findings for the last 4 observations

Answer: A student with a gre score of 800 and a 4.0 gpa has a 73.4% chance of being admitted if they attended a prestige 1 school. A student with the same scores having attended a prestige 2 school would have a 58.2% chance of admittance, while a student at a teir three and teir 4 school would have a 41.9% and 36.8% chance, respectively, when gre and gpa are equal.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.